In [1]:
import open3d as o3d
import numpy as np
import pandas as pd
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def write_ply_with_block_id(filename, points, colors=None, block_id=0):
    """
    Writes a PLY file with an extra scalar field 'Block_ID'.
    """
    with open(filename, "w") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points)}\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        if colors is not None:
            f.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
        f.write("property float Block_ID\n")
        f.write("end_header\n")

        if colors is not None:
            cols = (colors * 255).astype(np.uint8)
            for (x, y, z), (r, g, b) in zip(points, cols):
                f.write(f"{x} {y} {z} {r} {g} {b} {block_id}\n")
        else:
            for (x, y, z) in points:
                f.write(f"{x} {y} {z} {block_id}\n")

In [3]:
input_files = ["Campus_BlockBa.ply", "Campus_BlockBb.ply", "Campus_BlockBc.ply"]   # <-- your source file
output_dir = "Campus_Partitions"
voxel_size = 20.0   # tile size in meters
overlap = 0.5       # meters overlap between tiles
csv_path = os.path.join(output_dir, "partition_summary.csv")

os.makedirs(output_dir, exist_ok=True)

for input_file in input_files:
    # === Read point cloud ===
    print(f"Reading: {input_file}")
    pcd = o3d.io.read_point_cloud(input_file)
    pts = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None
    
    print(f"Total points: {len(pts):,}")
    min_bound = pts.min(axis=0)
    max_bound = pts.max(axis=0)
    print(f"Bounding box: {min_bound} → {max_bound}")

    summary = []
    
    nx = int(np.ceil((max_bound[0] - min_bound[0]) / voxel_size))
    ny = int(np.ceil((max_bound[1] - min_bound[1]) / voxel_size))
    print(f"Dividing into {nx} × {ny} tiles...")
    
    block_id = 0
    for ix in range(nx):
        for iy in range(ny):
            x_min = min_bound[0] + ix * voxel_size - overlap
            x_max = min_bound[0] + (ix + 1) * voxel_size + overlap
            y_min = min_bound[1] + iy * voxel_size - overlap
            y_max = min_bound[1] + (iy + 1) * voxel_size + overlap
    
            mask = (
                (pts[:, 0] >= x_min) & (pts[:, 0] <= x_max) &
                (pts[:, 1] >= y_min) & (pts[:, 1] <= y_max)
            )
    
            if np.sum(mask) == 0:
                continue
    
            block_id += 1
            sub_points = pts[mask]
            sub_colors = colors[mask] if colors is not None else None
    
            out_path = os.path.join(output_dir, f"{input_file[:-4]}_{block_id}.ply")
            write_ply_with_block_id(out_path, sub_points, sub_colors, block_id)
    
            file_size_mb = os.path.getsize(out_path) / (1024**2)
            num_points = len(sub_points)
    
            summary.append({
                "Block_ID": block_id,
                "File_Name": os.path.basename(out_path),
                "Num_Points": num_points,
                "Size_MB": round(file_size_mb, 2),
                "X_min": round(x_min, 3),
                "X_max": round(x_max, 3),
                "Y_min": round(y_min, 3),
                "Y_max": round(y_max, 3)
            })
            print(f"Saved {out_path}: {num_points:,} points (~{file_size_mb:.2f} MB)")
    
    pd.DataFrame(summary).to_csv(csv_path[:-4] + input_file[:-4] + ".csv", index=False)
    print(f"\n✅ Partition summary saved to {csv_path}")

Reading: Campus_BlockBa.ply
Total points: 313,879,356
Bounding box: [-48.373978 -99.885841  -6.608385] → [63.506924 12.118363 26.979216]
Dividing into 6 × 6 tiles...
Saved Campus_Partitions/Campus_BlockBa_1.ply: 1,425 points (~0.04 MB)
Saved Campus_Partitions/Campus_BlockBa_2.ply: 5,038,967 points (~158.72 MB)
Saved Campus_Partitions/Campus_BlockBa_3.ply: 8,848,611 points (~278.02 MB)
Saved Campus_Partitions/Campus_BlockBa_4.ply: 55,910 points (~1.72 MB)
Saved Campus_Partitions/Campus_BlockBa_5.ply: 10,111,771 points (~315.25 MB)
Saved Campus_Partitions/Campus_BlockBa_6.ply: 37,557,052 points (~1177.26 MB)
Saved Campus_Partitions/Campus_BlockBa_7.ply: 25,900,829 points (~811.33 MB)
Saved Campus_Partitions/Campus_BlockBa_8.ply: 1,060,550 points (~31.72 MB)
Saved Campus_Partitions/Campus_BlockBa_9.ply: 132,359 points (~4.02 MB)
Saved Campus_Partitions/Campus_BlockBa_10.ply: 18,001,184 points (~553.52 MB)
Saved Campus_Partitions/Campus_BlockBa_11.ply: 39,079,511 points (~1210.45 MB)
Saved

In [4]:
def read_ply(file_name):
    vertices = []
    with open(file_name, 'r') as file:
        lines = file.readlines()
        header_end = 0
        for i, line in enumerate(lines):
            if line.strip() == 'end_header':
                header_end = i + 1
                break
        vertex_lines = lines[header_end:]
        for line in vertex_lines:
            parts = line.split()
            x, y, z = map(float, parts[:3])  # x, y, z coordinates
            vertices.append((x, y, z))
    return vertices

In [5]:
def write_ply(file_name, vertices, block_id):
    # Start writing the header
    with open(file_name, 'w') as file:
        file.write('ply\n')
        file.write('format ascii 1.0\n')
        file.write(f'element vertex {len(vertices)}\n')
        file.write('property float x\n')
        file.write('property float y\n')
        file.write('property float z\n')
        file.write('property float scalar_intensity\n')
        file.write('property float scalar_blockid\n')
        file.write('end_header\n')

        # Write the vertices with the new properties
        for (x, y, z) in vertices:
            file.write(f'{x} {y} {z} 1 {block_id}\n')

In [6]:
csv_files = ["partition_summary_Campus_BlockBa.csv",  "partition_summary_Campus_BlockBb.csv",  "partition_summary_Campus_BlockBc.csv"]
for file_name in csv_files:
    csv_file = os.path.join(output_dir, file_name)
    suffix = csv_file[-5:-4]
    df = pd.read_csv(csv_file)
    grouped_files = []
    current_group = []
    current_size = 0
    
    # Block ID counter
    block_id_counter = 1
    
    # Iterate through the CSV rows to group by file size under 1 GB
    for _, row in df.iterrows():
        block_id = row['Block_ID']
        file_name = row['File_Name']
        size_mb = row['Size_MB']
    
        if current_size + size_mb <= 1024:  # If the cumulative size is less than 1GB
            current_group.append(file_name)
            current_size += size_mb
        else:
            # Process the current group of files
            grouped_files.append(current_group)
            
            # Reset the current group and size for the next one
            current_group = [file_name]
            current_size = size_mb
    
    # Don't forget to add the last group
    if current_group:
        grouped_files.append(current_group)
    
    # Step 5: Process each group of files
    output_dir_2 = 'grouped_ply_files'
    os.makedirs(output_dir_2, exist_ok=True)
    
    for group in grouped_files:
        all_vertices = []
        
        # Read all the PLY files in the current group and accumulate vertices
        for file_name in group:
            ply_vertices = read_ply(file_name)
            all_vertices.extend(ply_vertices)
    
        # Remove duplicates based on x, y, z
        unique_vertices = list(set(all_vertices))  # Remove duplicate vertices based on x, y, z
    
        # Step 6: Write the unique vertices to a new PLY file with a new Block ID
        new_file_name = os.path.join(output_dir, output_dir_2, f"Campus_BlockB{suffix}_group_{block_id_counter}.ply")
        write_ply(new_file_name, unique_vertices, block_id_counter)
        
        # Increment the block ID counter for the next file group
        block_id_counter += 1
    
    print(f"Grouped PLY files have been written to the directory: {output_dir}")

FileNotFoundError: [Errno 2] No such file or directory: 'Campus_Partitions/partition_summary_Campus_BlockBa.csv'